<a href="https://colab.research.google.com/github/ryderwishart/biblical-machine-learning/blob/main/semantic_domains_greek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
!pip install transformers
!pip install datasets
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=1c8dfef2da4a4

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments

import unicodedata
from google.colab import drive
import pandas as pd
import re
import torch
from datasets import load_dataset
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
from transformers import EarlyStoppingCallback
from collections import Counter
import json

drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


# Initialize Data

In [4]:
sentences_path = '/content/MyDrive/MyDrive/ln_domains_by_sent-unlemmatized.json'

In [5]:
# def strip_accents_and_lowercase(s):
#    return ''.join(c for c in unicodedata.normalize('NFD', s)
#                   if unicodedata.category(c) != 'Mn').lower()

In [6]:
domains_data = load_dataset("json", 
                            download_mode="force_redownload", 
                            data_files=sentences_path,
                            field="data"
                            )
# with open(sentences_path, 'r') as f:
#     data = f.read()

# raw_sentences = data.strip().split('\n\n')
# print(raw_sentences[0])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d32a7bf0b80d51b4/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Split the dataset into train, test, and evaluation sets
dataset_split = domains_data['train']
dataset = dataset_split.train_test_split(test_size=0.2, shuffle=True)
train_dataset, non_train_dataset = dataset['train'], dataset['test'].train_test_split(test_size=0.5, shuffle=True)
test_dataset, eval_dataset = non_train_dataset['train'], non_train_dataset['test']

# Print the sizes of the train, test, and evaluation sets
print(f"Train dataset size: {train_dataset}", )
print(f"Test dataset size: {test_dataset}")
print(f"Eval dataset size: {eval_dataset}")

Train dataset size: Dataset({
    features: ['tokens', 'tags'],
    num_rows: 6408
})
Test dataset size: Dataset({
    features: ['tokens', 'tags'],
    num_rows: 801
})
Eval dataset size: Dataset({
    features: ['tokens', 'tags'],
    num_rows: 802
})


In [8]:
# Create tag-to-id mapping dictionaries
all_tags = list(set([tag for tagset in domains_data['train']['tags'] for tag in tagset]))
all_tags = ['<pad>'] + all_tags
tag2idx = {tag:idx for idx, tag in enumerate(all_tags)}
idx2tag = {idx:tag for idx, tag in enumerate(all_tags)}

# train_dataset.add_column("_tags", train_dataset['tags']).rename_column("tags", "labels").add_column("tags", [tag2idx[tag] for tag in train_dataset['tags']]) #.cast(train_dataset.features)
# test_dataset.add_column("_tags", test_dataset['tags']).rename_column("tags", "labels").add_column("tags", [tag2idx[tag] for tag in test_dataset['tags']]) #.cast(test_dataset.features)
# eval_dataset.add_column("_tags", eval_dataset['tags']).rename_column("tags", "labels").add_column("tags", [tag2idx[tag] for tag in eval_dataset['tags']]) #.cast(eval_dataset.features)


In [9]:
def map_tags(example):
    # Map each tag in the example to its corresponding ID using tag2idx
    tags = [tag2idx[tag] for tag in example['tags']]
    labels = [idx2tag[tag] for tag in tags]
    # Return the example with the mapped tags
    assert len(labels) == len(tags)
    return {'labels': labels, 'tags': tags}

# Map the tags in the 'train' split
train_dataset = train_dataset.map(map_tags)

# Map the tags in the 'validation' split
eval_dataset = eval_dataset.map(map_tags)

# Map the tags in the 'test' split
test_dataset = test_dataset.map(map_tags)

Map:   0%|          | 0/6408 [00:00<?, ? examples/s]

Map:   0%|          | 0/802 [00:00<?, ? examples/s]

Map:   0%|          | 0/801 [00:00<?, ? examples/s]

In [10]:
from datasets import DatasetDict

raw_datasets = DatasetDict(
    {"train": train_dataset, "test": test_dataset, "eval": eval_dataset})

for i in zip(raw_datasets['test']['tags'][1], raw_datasets['test']['labels'][1]):
  print('id: ', i[0], 'label: ', i[1])

print(raw_datasets['test']['tokens'][1])
print(raw_datasets['test']['labels'][1])

id:  5787 label:  89.92
id:  6235 label:  90.90
id:  1266 label:  92.11
id:  4972 label:  19.4
['καὶ', 'ἐδίδοσαν', 'αὐτῷ', 'ῥαπίσματα.']
['89.92', '90.90', '92.11', '19.4']


In [11]:
print(raw_datasets['test']['labels'][0])

['89.124', '61.16', '15.77', '60.11', '33.69', '92.29', '33.69', '74.5', '20.54', '92.24', '7.15', '92.24', '12.1', '89.87', '67.136', '60.12', '67.186', '45.1']


# Initialize Model and Tokenizer

In [12]:
from transformers import file_utils
print('Models stored in cache at ', file_utils.default_cache_path)
import os

for i in os.listdir(file_utils.default_cache_path):
  print(i)

Models stored in cache at  /root/.cache/huggingface/hub
version.txt


In [13]:
pretrained = "pranaydeeps/Ancient-Greek-BERT"

In [14]:
# import shutil
# shutil.rmtree(pretrained)

FileNotFoundError: ignored

In [15]:
# Load the tokenizer and encode the input sentences
tokenizer = AutoTokenizer.from_pretrained(pretrained, force_download=True)
print(f'Tokenizer is fast: {tokenizer.is_fast}')

Tokenizer is fast: True


In [16]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            # print('>> cur word: ', current_word)
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
            prev_label = label
        elif word_id is None:
            # Special token
            # print('>> none word: ', word_id)
            new_labels.append(-100)
        else:
            # Same word as previous token
            # print('>> same as last word: ', word_id)
            label = labels[word_id]

            # If the label is B-XXX we change it to I-XXX
            # if label % 2 == 1:
            #     label += 1
            new_labels.append(label)

    return new_labels

In [17]:
inputs = tokenizer(raw_datasets["train"][1]["tokens"], is_split_into_words=True)
# inputs.tokens()

labels = raw_datasets["train"][1]["tags"]
strings = raw_datasets["train"][1]["tokens"]
word_ids = inputs.word_ids()
print(word_ids)
print(strings)

print(labels)
print(align_labels_with_tokens(labels, word_ids))

[None, 0, 1, 1, 1, 1, 2, 3, 3, 4, 5, 6, 7, 7, 8, 9, 10, 10, 10, 11, 12, 13, 14, 14, 15, 16, 17, 18, 18, 18, None]
['καὶ', 'ἐπολέμησεν', 'ὁ', 'δράκων', 'καὶ', 'οἱ', 'ἄγγελοι', 'αὐτοῦ,', 'καὶ', 'οὐκ', 'ἴσχυσαν,', 'οὐδὲ', 'τόπος', 'αὐτῶν', 'εὑρέθη', 'ἔτι', 'ἐν', 'τῷ', 'οὐρανῷ.']
[5748, 134, 2064, 1248, 5748, 2064, 2536, 5748, 5748, 5748, 5748, 2434, 6590, 5748, 2874, 6052, 5748, 2064, 6892]
[-100, 5748, 134, 134, 134, 134, 2064, 1248, 1248, 5748, 2064, 2536, 5748, 5748, 5748, 5748, 5748, 5748, 5748, 2434, 6590, 5748, 2874, 2874, 6052, 5748, 2064, 6892, 6892, 6892, -100]


In [18]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding='max_length', max_length=42
    )
    all_labels = examples["tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        # print('>>>', labels)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [19]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/6408 [00:00<?, ? examples/s]

Map:   0%|          | 0/801 [00:00<?, ? examples/s]

Map:   0%|          | 0/802 [00:00<?, ? examples/s]

In [25]:
tag2idx['2.11']

2

In [55]:
tokenized_datasets['train']['labels'][0]

[-100,
 4992,
 3696,
 3696,
 2625,
 613,
 846,
 846,
 4703,
 6973,
 6973,
 6973,
 2672,
 6521,
 6521,
 5394,
 613,
 1489,
 1489,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100]

In [26]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [27]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100, 5748, 3594, 3594, 2064, 1300, 1300, 5748, 2064, 7457, 7457, 2064,
         4371, 4371, 4371, 4371, 5748, 5748, 2064, 4383, 4383, 4383, 5748, 1826,
         1826, 1826, 2064, 4383, 4383, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100],
        [-100, 5748,  134,  134,  134,  134, 2064, 1248, 1248, 5748, 2064, 2536,
         5748, 5748, 5748, 5748, 5748, 5748, 5748, 2434, 6590, 5748, 2874, 2874,
         6052, 5748, 2064, 6892, 6892, 6892, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100]])

# Evaluate model training

In [29]:
# !pip install seqeval
!pip install evaluate
import evaluate

metric = evaluate.load("seqeval")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.3 MB/s eta 0:00:00


In [30]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[all_tags[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [all_tags[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    ###### MAJOR GOTCHA: the module sequence_labeling.py from seqeval expects
    ###### named-entity tags with a certain format. This will raise an exception
    ###### in between epochs if you don't modify that code accordingly.
    ###### TODO: update this compute_metrics function to use sklearn as the
    ###### commented code block below does (though the block below is for single-
    ###### token inputs).
    
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }



# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# def compute_metrics(p):
#     # FIXME: I think I need to handle the p variable here slightly differently as it might lock up a lot of GPU ram to access it like this
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
#     precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
#     f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [31]:
# idx2tag

In [32]:
# Define the model architecture
defined_model = AutoModelForTokenClassification.from_pretrained(
    pretrained,
    id2label=idx2tag,
    label2id=tag2idx,
    force_download=True
)

# defined_model = defined_model.to('cuda')

assert defined_model.config.num_labels == len(all_tags)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at pranaydeeps/Ancient-Greek-BERT and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train Model

In [33]:
from transformers import TrainingArguments

batch_size = 2

args = TrainingArguments(
    pretrained,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    # push_to_hub=True,
    per_device_train_batch_size=batch_size, # optional, defaults to 8 — The batch size per GPU/TPU core/CPU for training.
    per_device_eval_batch_size=batch_size
)

In [ ]:
# optionally clear GPU cache before training if you are getting a CUDA out of memory error
# torch.no_grad()
# !pip install numba

# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()

In [33]:
# Use a subset for testing

# subset_dataset_train = tokenized_datasets['train'].select(range(100))
# subset_dataset_test = tokenized_datasets['test'].select(range(100))
# subset_dataset_eval = tokenized_datasets['eval'].select(range(100))
# trainer = Trainer(
#     model=defined_model,
#     args=args,
#     # train_dataset=tokenized_datasets["train"],
#     train_dataset=subset_dataset_train,
#     # eval_dataset=tokenized_datasets["test"], # use 'test' for training evaluation
#     eval_dataset=subset_dataset_test, # use 'test' for training evaluation
#     data_collator=data_collator,
#     # compute_metrics=compute_metrics,
#     tokenizer=tokenizer,
# )

In [34]:
from transformers import Trainer

trainer = Trainer(
    model=defined_model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"], # use 'test' for training evaluation
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6408
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 9612
  Number of trainable parameters = 118342025


Epoch,Training Loss,Validation Loss
1,3.716600,3.411424
2,2.769100,2.838049
3,2.388400,2.704520


***** Running Evaluation *****
  Num examples = 801
  Batch size = 2
Saving model checkpoint to pranaydeeps/Ancient-Greek-BERT/checkpoint-3204
Configuration saved in pranaydeeps/Ancient-Greek-BERT/checkpoint-3204/config.json
Model weights saved in pranaydeeps/Ancient-Greek-BERT/checkpoint-3204/pytorch_model.bin
tokenizer config file saved in pranaydeeps/Ancient-Greek-BERT/checkpoint-3204/tokenizer_config.json
Special tokens file saved in pranaydeeps/Ancient-Greek-BERT/checkpoint-3204/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 801
  Batch size = 2
Saving model checkpoint to pranaydeeps/Ancient-Greek-BERT/checkpoint-6408
Configuration saved in pranaydeeps/Ancient-Greek-BERT/checkpoint-6408/config.json
Model weights saved in pranaydeeps/Ancient-Greek-BERT/checkpoint-6408/pytorch_model.bin
tokenizer config file saved in pranaydeeps/Ancient-Greek-BERT/checkpoint-6408/tokenizer_config.json
Special tokens file saved in pranaydeeps/Ancient-Greek-BERT/checkpoint-640

TrainOutput(global_step=9612, training_loss=3.353432428722723, metrics={'train_runtime': 931.1908, 'train_samples_per_second': 20.645, 'train_steps_per_second': 10.322, 'total_flos': 441170668550304.0, 'train_loss': 3.353432428722723, 'epoch': 3.0})

In [35]:
# !huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [36]:
# !pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
# defined_model.push_to_hub("semdom-ancient-greek")

Configuration saved in /tmp/tmp5nc9ue6t/config.json
Model weights saved in /tmp/tmp5nc9ue6t/pytorch_model.bin
Uploading the following files to ryderwishart/semdom-ancient-greek: pytorch_model.bin,config.json


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/473M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ryderwishart/semdom-ancient-greek/commit/df8f43415e35694cba84b7305fab427140aaab2b', commit_message='Upload BertForTokenClassification', commit_description='', oid='df8f43415e35694cba84b7305fab427140aaab2b', pr_url=None, pr_revision=None, pr_num=None)

# Evaluate Model

In [53]:
# TODO: use programmatic evaluation; implement correct compute_metrics function above
from transformers import pipeline

model_checkpoint = defined_model.cpu()
token_classifier = pipeline(
    "token-classification", 
    model=model_checkpoint, 
    tokenizer=tokenizer, 
    aggregation_strategy="simple"
)
output = token_classifier("Λέγει αὐτῷ Θωμᾶς Κύριε, οὐκ οἴδαμεν ποῦ ὑπάγεις· πῶς οἴδαμεν τὴν ὁδὸν;")
test = {"tokens":["Λέγει","αὐτῷ","Θωμᾶς","Κύριε,","οὐκ","οἴδαμεν","ποῦ","ὑπάγεις·","πῶς","οἴδαμεν","τὴν","ὁδὸν;"],"tags":["33.69","92.11","93.155","12.9{N:001}","69.3","28.1","83.6","15.35","92.16","NULL","92.24","1.99"]}
for i, tok in enumerate(output):
    print(tok['word'], tok['entity_group'] == test['tags'][i], '\t\tpred: ', tok['entity_group'], '\t\tshould be: ', test['tags'][i])

λεγει True 		pred:  33.69 		should be:  33.69
αυτω True 		pred:  92.11 		should be:  92.11
θωμας False 		pred:  93.294a 		should be:  93.155
κυριε, True 		pred:  12.9{N:001} 		should be:  12.9{N:001}
ουκ True 		pred:  69.3 		should be:  69.3
οιδαμεν True 		pred:  28.1 		should be:  28.1
που False 		pred:  92.14 		should be:  83.6
υπαγεις · False 		pred:  15.37 		should be:  15.35
πως True 		pred:  92.16 		should be:  92.16
οιδαμεν False 		pred:  28.1 		should be:  NULL
την True 		pred:  92.24 		should be:  92.24
οδον ; True 		pred:  1.99 		should be:  1.99
